#**Tugas 2 Jobsheet 7**

In [3]:
!pip install annoy faiss-cpu hnswlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 23.7 MB/s eta 0:00:00
  Created wheel for annoy: filename=annoy-1.17.3-cp312-cp312-linux_x86_64.whl size=551809 sha256=fff786b3290ce2ba7328c927fc99e3da936530cf97ff2a4ec6c83da9d381aad2
  Stored in directory: /root/.cache/pip/wheels/db/b9/53/a3b2d1fe1743abadddec6aa541294b24fdbc39d7800bc57311
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp312-cp312-linux_x86_64.whl size=2528145 sha256=f1e45a9af8f7d149e11447ff6991a14586064b9900620d44416603603bf60c23
  Stored in directory: /root/.cache/pip/wheels/ac/39/b3/cbd7f9cbb76501d2d5fbc84956e70d0b94e788aac87bda465e
Successfully built annoy hnswlib


In [5]:


import time
import numpy as np
import pandas as pd
import os
import gc
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors


# -----------------------
# Konfigurasi percobaan
# -----------------------
CSV_PATH = "/content/drive/MyDrive/Colab Notebooks/Pembelarajan Mesin/Minggu 7/songs_with_attributes_and_lyrics.csv"   # ganti sesuai path Anda
FEATURES = ['danceability', 'energy', 'loudness', 'speechiness',
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

k = 10               # hitung nearest neighbors @k
n_queries = 1000     # jumlah query acak (ubah sesuai ukuran dataset / resource)
use_subset = None    # jika ingin uji pada subset data set e.g. 50000 -> set int, atau None = pakai semua baris

# -----------------------
# 1) Load dataset & preprocessing
# -----------------------
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"File '{CSV_PATH}' tidak ditemukan. Letakkan CSV yang Anda unduh sebagai '{CSV_PATH}'")

print("Membaca CSV...")
df = pd.read_csv(CSV_PATH)

# Pastikan fitur ada; jika tidak, pilih semua numeric
missing = [f for f in FEATURES if f not in df.columns]
if missing:
    print("Kolom yang diminta tidak semua ada. Mengambil semua kolom numerik yang tersedia.")
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    # filter out id/unncessary columns? we'll use numeric_cols
    FEATURES = numeric_cols
else:
    # tetap gunakan FEATURES user-specified
    pass

# Drop rows with NaN pada fitur yang digunakan
df_feat = df[FEATURES].copy()
before = len(df_feat)
df_feat = df_feat.dropna(axis=0, how='any')
after = len(df_feat)
print(f"Dropped {before-after} rows karena NaN. Sisa baris: {after}")

# Optionally take subset (faster testing)
if use_subset is not None and use_subset < len(df_feat):
    df_feat = df_feat.sample(n=use_subset, random_state=RANDOM_SEED).reset_index(drop=True)

X = df_feat.values.astype('float32')  # faiss expects float32
n_samples, dim = X.shape
print(f"Dataset: {n_samples} samples, dim = {dim}")

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X).astype('float32')

# Build query set (random indices)
if n_queries > n_samples:
    n_queries = n_samples
query_idx = np.random.choice(n_samples, size=n_queries, replace=False)
X_queries = X_scaled[query_idx]

# For evaluation: compute exact NN using sklearn NearestNeighbors (brute force L2)
print("Membangun exact (brute-force) reference dengan sklearn NearestNeighbors...")
t0 = time.time()
nn_brute = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean').fit(X_scaled)
dist_ref, idx_ref = nn_brute.kneighbors(X_queries, return_distance=True)
t_brute_build = time.time() - t0
# query timing for brute force measured separately
tq0 = time.time()
dist_ref2, idx_ref2 = nn_brute.kneighbors(X_queries, return_distance=True)
t_brute_query = (time.time() - tq0) / n_queries
print(f"Brute build time: {t_brute_build:.3f}s, avg query time: {t_brute_query*1000:.4f} ms/query")

# Utility: function to compute recall@k and avg query time
def evaluate_index(query_func, name):
    # query_func should accept (queries, k) and return (indices, distances) arrays
    gc.collect()
    t0 = time.time()
    inds, dists = query_func(X_queries, k)
    query_time = (time.time() - t0) / n_queries
    # compute recall@k: for each query, overlap between returned set and reference set
    correct = 0
    for i in range(len(inds)):
        set_ref = set(idx_ref[i])
        set_idx = set(inds[i])
        correct += len(set_ref.intersection(set_idx)) / float(k)
    recall_at_k = correct / len(inds)
    print(f"{name}: avg query time = {query_time*1000:.4f} ms/query, recall@{k} = {recall_at_k:.4f}")
    return {"name": name, "avg_query_ms": query_time*1000, "recall_at_k": recall_at_k}

results = []

# -----------------------
# 2) ANNOY
# -----------------------
try:
    print("\n--- ANNOY ---")
    from annoy import AnnoyIndex
    metric = 'euclidean'  # options: 'angular' 'euclidean' 'manhattan' 'hamming' 'dot'
    ann = AnnoyIndex(dim, metric)
    # build index
    t0 = time.time()
    for i, v in enumerate(X_scaled):
        ann.add_item(i, v)
    n_trees = 50   # tradeoff: lebih banyak trees -> lebih akurat tapi lebih lambat/dan memori
    ann.build(n_trees)
    t_build = time.time() - t0
    print(f"Annoy build time: {t_build:.3f}s (n_trees={n_trees})")

    def annoy_query(queries, kk):
        inds = []
        dists = []
        for q in queries:
            ids, distances = ann.get_nns_by_vector(q.tolist(), kk, include_distances=True)
            inds.append(ids)
            dists.append(distances)
        return np.array(inds, dtype=int), np.array(dists, dtype=float)

    res_annoy = evaluate_index(annoy_query, "Annoy")
    res_annoy["build_time_s"] = t_build
    res_annoy["params"] = {"n_trees": n_trees, "metric": metric}
    results.append(res_annoy)
except Exception as e:
    print("Error running Annoy:", e)

# -----------------------
# 3) FAISS
# -----------------------
try:
    print("\n--- FAISS (CPU) ---")
    import faiss
    # 3a) Exact Faiss (IndexFlatL2) for baseline exact (fast brute L2)
    t0 = time.time()
    index_flat = faiss.IndexFlatL2(dim)  # exact
    index_flat.add(X_scaled)
    t_build_flat = time.time() - t0
    print(f"Faiss IndexFlatL2 build time: {t_build_flat:.3f}s")
    # query function
    def faiss_flat_query(queries, kk):
        # faiss expects float32 numpy arrays
        D, I = index_flat.search(queries.astype('float32'), kk)
        return I, D

    res_faiss_exact = evaluate_index(faiss_flat_query, "Faiss(IndexFlatL2) exact")
    res_faiss_exact["build_time_s"] = t_build_flat
    results.append(res_faiss_exact)

    # 3b) Approx Faiss: IVF + PQ or IVF Flat (IndexIVFFlat)
    nlist = 100  # number of clusters
    quantizer = faiss.IndexFlatL2(dim)
    index_ivf = faiss.IndexIVFFlat(quantizer, dim, nlist, faiss.METRIC_L2)
    # training
    t0 = time.time()
    # need training vectors: choose up to 100k randomly or all
    ntrain = min(100000, n_samples)
    train_idx = np.random.choice(n_samples, ntrain, replace=False)
    xb_train = X_scaled[train_idx]
    index_ivf.train(xb_train)
    index_ivf.add(X_scaled)
    t_build_ivf = time.time() - t0
    index_ivf.nprobe = 10  # tradeoff: larger -> more accurate slower
    print(f"Faiss IndexIVFFlat build/train time: {t_build_ivf:.3f}s (nlist={nlist}, nprobe={index_ivf.nprobe})")

    def faiss_ivf_query(queries, kk):
        D, I = index_ivf.search(queries.astype('float32'), kk)
        return I, D

    res_faiss_ivf = evaluate_index(faiss_ivf_query, "Faiss(IndexIVFFlat) approx")
    res_faiss_ivf["build_time_s"] = t_build_ivf
    res_faiss_ivf["params"] = {"nlist": nlist, "nprobe": index_ivf.nprobe}
    results.append(res_faiss_ivf)

except Exception as e:
    print("Error running Faiss:", e)

# -----------------------
# 4) HNSWLIB
# -----------------------
try:
    print("\n--- HNSWLIB ---")
    import hnswlib
    p = hnswlib.Index(space='l2', dim=dim)  # use 'l2' metric for Euclidean
    M = 48          # tradeoff param (higher == more accuracy and memory)
    ef_construction = 200
    t0 = time.time()
    p.init_index(max_elements=n_samples, ef_construction=ef_construction, M=M)
    p.add_items(X_scaled, np.arange(n_samples))
    p.set_ef(50)   # query-time parameter: higher -> more accurate slower
    t_build_hnsw = time.time() - t0
    print(f"HNSWLIB build time: {t_build_hnsw:.3f}s (M={M}, ef_construction={ef_construction}), ef_query={p.get_ef()}")

    def hnsw_query(queries, kk):
        labels, distances = p.knn_query(queries, k=kk)
        return labels, distances

    res_hnsw = evaluate_index(hnsw_query, "HNSWLIB")
    res_hnsw["build_time_s"] = t_build_hnsw
    res_hnsw["params"] = {"M": M, "ef_construction": ef_construction, "ef_query": p.get_ef()}
    results.append(res_hnsw)

except Exception as e:
    print("Error running hnswlib:", e)

# -----------------------
# Summary
# -----------------------
print("\n=== SUMMARY ===")
import json
print(json.dumps(results, indent=2))

# Optionally save results
out_df = pd.DataFrame(results)
out_df.to_csv("ann_index_comparison_results.csv", index=False)
print("Hasil disimpan ke ann_index_comparison_results.csv")


Membaca CSV...
Dropped 0 rows karena NaN. Sisa baris: 955320
Dataset: 955320 samples, dim = 9
Membangun exact (brute-force) reference dengan sklearn NearestNeighbors...
Brute build time: 4.537s, avg query time: 6.0751 ms/query

--- ANNOY ---
Annoy build time: 91.639s (n_trees=50)
Annoy: avg query time = 0.3451 ms/query, recall@10 = 0.9927

--- FAISS (CPU) ---
Faiss IndexFlatL2 build time: 0.087s
Faiss(IndexFlatL2) exact: avg query time = 1.5554 ms/query, recall@10 = 0.9981
Faiss IndexIVFFlat build/train time: 0.328s (nlist=100, nprobe=10)
Faiss(IndexIVFFlat) approx: avg query time = 0.7234 ms/query, recall@10 = 0.9975

--- HNSWLIB ---
Error running hnswlib: 'hnswlib.Index' object has no attribute 'get_ef'

=== SUMMARY ===
[
  {
    "name": "Annoy",
    "avg_query_ms": 0.3451261520385742,
    "recall_at_k": 0.992699999999999,
    "build_time_s": 91.63931703567505,
    "params": {
      "n_trees": 50,
      "metric": "euclidean"
    }
  },
  {
    "name": "Faiss(IndexFlatL2) exact",
    